This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [10]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'hZgYBRB6zZsCSJ4Z5_tq'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [11]:
# First, import the relevant modules
import requests as rqs


In [12]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date={2016-09-19}&end_date={2019-09-19}&api_key=' + API_KEY 
data = rqs.get(url).json()

In [13]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data.keys()



dict_keys(['dataset'])

We can see that the file is actually a dictionary that has only one key: `'dataset'`The value related with this key is another dictionary with the following keys:

In [14]:
#Keys that are inside the 'dataset' dictionary
data['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

We are interested in two particular keys: the `'data'` whose value contains the different prices as a list of lists and the `'column_names'`key, that is related to a list that classifies the data and let us know what type of price or information is presented in the `'data'` list. 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

First, we read the required data. We already know that the file that is downloaded is presented as a dictionary with only one key: the `'dataset'` dictionary in which we will find two important entries: the `'column_names'` and the `'data'` elements. First we will show the column names and will assign them to the variable `col`:

In [15]:
#Reading 2017 data
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date={2017-01-01}&end_date={2017-12-31}&api_key=' + API_KEY 
data = rqs.get(url).json()
col = data['dataset']['column_names']
col



['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

We can see that, for every day, the file has eleven different values: `'Date'`, `'Open'`, `'High'`, `'Low'`, `'Close'`, `'Change'`, `'Traded Volume'`, `'Turnover'`, `'Last Price of the Day'`, `'Daily Traded Units'`, and `'Daily Turnover'`.

First, we will use this `column_names` entry to create an index, afterwards, we will extract the `data` list.

Now, let us take a look at the data, we will use the variable `d` to store this information. As we can see below, `d` is a list that has more lists inside, one list per day. We show the last four days of december below:

In [16]:
#We get the data list from the 'dataset' dictionary
d = data['dataset']['data']
d[:4]

[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None],
 ['2017-12-27',
  51.45,
  51.89,
  50.76,
  51.82,
  None,
  57452.0,
  2957018.0,
  None,
  None,
  None],
 ['2017-12-22',
  51.05,
  51.5,
  50.92,
  51.32,
  None,
  71165.0,
  3641949.0,
  None,
  None,
  None]]

It appears we have some values that are registered with the legend "None". Apparently, those values are `'Change'`, `'Last Price of the Day'`, `'Daily Traded Units'`, and `'Daily Turnover'`. Before analyzing anything, we must create an index that help us navigate in the data.

In [66]:
#Index variable
ind = dict(zip(data['dataset']['column_names'], range(len(col))))

Now, we can use the index to count the number of 'None' values and its location:

In [22]:
none = {i:len([x[ind[i]] for x in d if x[ind[i]] != None]) for i in col}
none

{'Date': 255,
 'Open': 252,
 'High': 255,
 'Low': 255,
 'Close': 255,
 'Change': 1,
 'Traded Volume': 255,
 'Turnover': 255,
 'Last Price of the Day': 0,
 'Daily Traded Units': 0,
 'Daily Turnover': 0}

We can see that there are some values that are registered as "None" for each day. Such is the case of `'Last Price of the Day'`, `'Daily Traded Units'`, and `'Daily Turnover'`. On the other hand, we an see that the `'Open'` column has three "None" values. It is interesting to see that the `'Change'`column has only one value that is not "None". We must see to what date it belongs to.

In [48]:
NegNone = [y for y in [x[ind['Change']] for x in d] if y != None]
v = [x[ind['Change']] for x in d].index(NegNone[0])
[x[ind['Date']] for x in d][v]



'2017-05-01'

The date above corresponds to the only non-Null value.

We will now write a set of functions in order to get the requested values. The main function will be called `analysis` and the return value will be a dictionary.

In [59]:
def median(x):
    if len(x)%2:
        m = x[int(len(x)/2)]
    else:
        m = (x[int(len(x)/2-1)] + x[int(len(x)/2)])/2
    return m

#List creator: choose the column from which you want to create a list.

def analysis(value, month1, month2,days,wanted):
    """Function that computes Highest, lowest values, average and median for a specific time
    inerval defined by de user.
    Choose  Column name, first month, second month and space between days for analysis"""
    
    d2 = [x for x in d if (int(x[0][5:7])>=month1 and int(x[0][5:7])<=month2)]
    dates = [a[ind['Date']] for a in d2]
    values = [b[ind[value]]for b in d2]
    
    nulls = [dates[i] for i in range(len(values)) if values[i] == None]
    nnvalues = [x for x in values if x != None]
    changes = [nnvalues[i]-nnvalues[i-days] for i in range(days,len(nnvalues))]
    
    res = {'Name':value, 'Number of nulls':len(nulls), 'Number of  analyzed days':len(nnvalues), 'From':dates[0], 'To':dates[-1],
    'Max':max(nnvalues), 'Min':min(nnvalues), 'Average':sum(nnvalues)/len(nnvalues), 'Median':median(nnvalues),
    'Max_Change':max(changes), 'Min_change':min(changes)}
    
    return {i:res[i] for i in wanted}


The functon above will allow us to compute each of the requested values.
We must choose the values that will be presented: Such values are: 'Max', 'Min', 'Average', 'Median', 'Max_Change', 'Min_Change'

## Highest and lowest opening prices:

We will use our main function:

In [60]:
analysis('Open', 1,12, 1,['Max','Min'])

{'Max': 53.11, 'Min': 34.0}

##  Largest change in one day in High and Low Prices:

We must use the `analysis` function with the options "High" and "Low" respectively:



In [61]:
analysis('High',1,12,1,['Max_Change'])

{'Max_Change': 2.809999999999995}

In [62]:
analysis('Low', 1,12,1,['Max_Change'])

{'Max_Change': 3.4399999999999977}

## Largest change between any two days in Closing Price

This time, we use the analysis function choosing the option "Close" and changing the number of days to 2.

In [63]:
analysis('Close',1,12,2,['Max_Change'])

{'Max_Change': 3.1499999999999986}

## Average daily trading volume and median during this year

We choose the option "Traded Volume" in the function and also choose the average and median options.

In [75]:
analysis('Traded Volume',12,12,1,['Average','Median'])

{'Average': 94345.94736842105, 'Median': 132981.0}